# 路径

In [2]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")

projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)

D:\project_meta\NNproject\NNI


In [3]:
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
pprint.pprint(datasetDict)
pprint.pprint(outputDict)

{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
{'ECG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'ECG_uniformfilted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'EEG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\EEG',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'EEG_uniformfilted_folder': 'D:\\project_

# 数据集

In [47]:
from project.dataset.ExtendDataset import *
import pandas as pd
from project.dataprocess.DatasetProcess import *
toRefreshWindowsTbale=False
if toRefreshWindowsTbale==True and os.path.exists(outputDict['windowsTable']):
    os.remove(outputDict['windowsTable'])
    windowsTable=MakeWindowsTable(psg_folder=datasetDict['psg_folder'],kss_file_path=datasetDict['kss_file'])
    windowsTable.to_csv(outputDict['windowsTable'])
if not os.path.exists(outputDict['windowsTable']):
    windowsTable=MakeWindowsTable(psg_folder=datasetDict['psg_folder'],kss_file_path=datasetDict['kss_file'])
    windowsTable.to_csv(outputDict['windowsTable'])

eegControlTable=pd.read_csv(outputDict['windowsTable'],index_col=0,dtype={'ID': int})
eegControlTable


,ID,LEVEL,KSS,START,END
0,1,1.0,3.0,0.0,15.0
1,1,1.0,3.0,15.0,30.0
2,1,1.0,3.0,30.0,45.0
3,1,1.0,3.0,45.0,60.0
4,1,1.0,3.0,60.0,75.0
...,...,...,...,...,...
1435,9,3.0,8.0,525.0,540.0
1436,9,3.0,8.0,540.0,555.0
1437,9,3.0,8.0,555.0,570.0
1438,9,3.0,8.0,570.0,585.0


In [57]:
import pandas as pd

# 假设您的数据在一个名为 df 的 DataFrame 中
df = pd.DataFrame({'id': [10, 20, 30, 40, 50],
                   'value': [1, 2, 3, 4, 5]})

# 使用布尔索引来排除 id 为 10 的数据
df_filtered = df[~df['id'].isin([10, 20])].reset_index(drop=True)


print(df_filtered)
print(df)

   id  value
0  30      3
1  40      4
2  50      5
   id  value
0  10      1
1  20      2
2  30      3
3  40      4
4  50      5


In [59]:
id = sorted(eegControlTable['ID'].unique())
print(id)
from sklearn.model_selection import KFold
kfold=KFold(n_splits=7)
KFOLD0=list(kfold.split(id))[0]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [62]:
print(KFOLD0[0])
trainControlTable=eegControlTable[eegControlTable['ID'].isin(KFOLD0[0])]
id = sorted(trainControlTable['ID'].unique())
print(id)

[ 2  3  4  5  6  7  8  9 10 11 12 13]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [63]:
testControlTable=eegControlTable[eegControlTable['ID'].isin(KFOLD0[1])]


In [72]:
EEGDataset=LEVELEEGEndToEndDataset(eegControlTable,datasetDict['psg_folder'],Mintime=15)
EEGDataset[0][0].shape

(5, 7680)

In [64]:
trainDataset=LEVELEEGEndToEndDataset(trainControlTable,datasetDict['psg_folder'],Mintime=15)
trainDataloader=DataLoader(trainDataset,batch_size=128)
testDataset=LEVELEEGEndToEndDataset(testControlTable,datasetDict['psg_folder'],Mintime=15)
testDataloader=DataLoader(testDataset,batch_size=128)

In [27]:
len(EEGDataset)

1368

In [7]:
print(EEGDataset[0][0].shape)
channels=EEGDataset[0][0].shape[0]
print("通道数为：",channels)
classes=3
print(f'类别数位： {classes}')

(5, 7680)
通道数为： 5
类别数位： 3


In [8]:
eegDataloader=DataLoader(EEGDataset,batch_size=128)

# 模型

In [28]:
from project.model.CNN1D import simple_cnn1d
x=next(iter(eegDataloader))
x

[tensor([[[-1.5625e-05, -1.3575e-05, -9.4729e-06,  ...,  1.0645e-05,
            7.4221e-06,  6.6408e-06],
          [-2.4708e-05, -2.3048e-05, -2.0020e-05,  ...,  1.0352e-05,
            1.0743e-05,  1.1524e-05],
          [-8.0081e-06, -3.8087e-06,  2.3438e-06,  ...,  1.4161e-05,
            1.4942e-05,  1.2403e-05],
          [-7.1291e-06, -3.9064e-06, -2.7345e-06,  ...,  8.9846e-06,
            9.4729e-06,  1.3672e-05],
          [-9.1800e-06, -7.6174e-06, -2.7345e-06,  ...,  1.3282e-05,
            1.3575e-05,  1.3965e-05]],
 
         [[ 4.7853e-06, -1.0743e-06, -6.6408e-06,  ..., -7.9104e-06,
           -5.3713e-06, -8.7893e-07],
          [ 9.6683e-06,  3.4181e-06, -3.3204e-06,  ..., -1.4649e-05,
           -1.2110e-05, -8.0081e-06],
          [ 7.1291e-06,  0.0000e+00, -7.5198e-06,  ...,  3.5157e-06,
            8.0081e-06,  1.1719e-05],
          [ 1.4454e-05,  8.3010e-06,  1.8555e-06,  ..., -7.0315e-06,
           -5.6642e-06, -3.3204e-06],
          [ 1.2110e-05,  6.4455e-0

In [29]:
from torchsummary import summary
Model=simple_cnn1d(channels,classes,[16,32,64])
summary(Model,(5,4*2560),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 16, 10240]             576
       BatchNorm1d-2            [-1, 16, 10240]              32
              ReLU-3            [-1, 16, 10240]               0
         MaxPool1d-4             [-1, 16, 5119]               0
            Conv1d-5             [-1, 32, 2560]           1,568
       BatchNorm1d-6             [-1, 32, 2560]              64
              ReLU-7             [-1, 32, 2560]               0
            Conv1d-8             [-1, 64, 1280]           6,208
       BatchNorm1d-9             [-1, 64, 1280]             128
             ReLU-10             [-1, 64, 1280]               0
AdaptiveAvgPool1d-11                [-1, 64, 1]               0
          Flatten-12                   [-1, 64]               0
             ReLU-13                   [-1, 64]               0
           Linear-14                   

In [12]:
Model(x[0].to(torch.float32))

tensor([[-0.0854,  0.1305,  0.1510],
        [-0.0839,  0.1065,  0.1445],
        [-0.0840,  0.0871,  0.1520],
        [-0.0884,  0.1193,  0.1551],
        [-0.0892,  0.1122,  0.1358],
        [-0.0989,  0.1283,  0.1346],
        [-0.1077,  0.1235,  0.1657],
        [-0.1032,  0.1299,  0.1393],
        [-0.0934,  0.1231,  0.1496],
        [-0.1013,  0.1229,  0.1420],
        [-0.0863,  0.1202,  0.1453],
        [-0.1098,  0.1290,  0.1558],
        [-0.0897,  0.1124,  0.1474],
        [-0.0901,  0.1217,  0.1568],
        [-0.0947,  0.1195,  0.1472],
        [-0.1044,  0.1279,  0.1510],
        [-0.0953,  0.1258,  0.1486],
        [-0.0983,  0.1190,  0.1526],
        [-0.0953,  0.1114,  0.1539],
        [-0.0594,  0.1057,  0.1508],
        [-0.0917,  0.1184,  0.1515],
        [-0.0966,  0.1198,  0.1447],
        [-0.0967,  0.1214,  0.1449],
        [-0.0972,  0.1128,  0.1532],
        [-0.1075,  0.1288,  0.1470],
        [-0.0891,  0.1200,  0.1446],
        [-0.0886,  0.1155,  0.1465],
 

In [13]:
from typing import Any, Optional
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT, TRAIN_DATALOADERS, OptimizerLRScheduler
from project.model.CNN1D import simple_cnn1d
from torchmetrics.functional import accuracy

class PLSimpleCNN1d(pl.LightningModule):
    
    def __init__(self, input_channels,num_classes,list_down:list,num_hidden=32,*args, **kwargs):
        super().__init__()
        self.Model=simple_cnn1d(input_channels,num_classes,list_down,num_hidden)
        self.LossFunction=torch.nn.CrossEntropyLoss()
        self.Optimizer=torch.optim.Adam(self.Model.parameters(),lr=0.01)
    def forward(self,x, *args: Any, **kwargs: Any) -> Any:
        y=self.Model(x)
        return y
    def training_step(self, batch,batch_idx,*args: Any, **kwargs: Any) -> STEP_OUTPUT:
        self.Optimizer.zero_grad()
        x,y=batch
        y_hat=Model(x.to(torch.float32))
        loss=self.LossFunction(y_hat,y.long()-1)
        return {'loss':loss}
    def configure_optimizers(self) -> OptimizerLRScheduler:
        return self.Optimizer
    # def test_step(self,batch,batch_idx, *args: Any, **kwargs: Any) -> STEP_OUTPUT | None:
    #     x,y=batch
    #     x=x.to(torch.float32)
    #     y_hat=self.Model(x)
    #     loss=self.LossFunction(y_hat,y)
    #     acc=accuracy(y_hat,y)
    #     return

        

In [69]:
from typing import Any, Optional
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT, TRAIN_DATALOADERS, OptimizerLRScheduler
from project.model.CNN1D import simple_cnn1d
from torchmetrics.functional import accuracy
from torch.utils.tensorboard import SummaryWriter

class TrainModel(pl.LightningModule):
    
    def __init__(self, net,lossFunction,optimizer,*args, **kwargs):
        super().__init__()
        self.Writer=SummaryWriter(log_dir=r"D:\project_meta\NNproject\NNI\output\tensorboar_log")
        self.Model=net
        self.LossFunction=lossFunction
        self.Optimizer=optimizer

    def forward(self,x, *args: Any, **kwargs: Any) -> Any:
        y=self.Model(x)
        return y
    def training_step(self, batch,batch_idx,*args: Any, **kwargs: Any) -> STEP_OUTPUT:
        self.Optimizer.zero_grad()
        x,y=batch
        y=y.long()-1
        
        y_hat=self.Model(x.to(torch.float32))
        loss=self.LossFunction(y_hat,y)
        # print(type(x),type(y),type(y_hat))
        # print(x.shape,y.shape,y_hat.shape)
        acc=accuracy(y_hat,y,task='MULTICLASS',num_classes=3)
        pbar={'train_acc':acc}
        # self.Writer.add_scalar('acc',acc,)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        # self.
        # print(acc)
        return {'loss':loss,'progress_bar':pbar}
    def configure_optimizers(self) -> OptimizerLRScheduler:
        return self.Optimizer
    def validation_step(self,batch,batch_idx, *args: Any, **kwargs: Any) -> STEP_OUTPUT | None:
        x,y=batch
        y=y.long()-1
        x=x.to(torch.float32)
        y_hat=self.Model(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='MULTICLASS',num_classes=3)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        return {'loss':loss}

        

# 训练

In [73]:
from pytorch_lightning import loggers
logger=loggers.TensorBoardLogger(save_dir=r'D:\project_meta\NNproject\NNI\notebook\TraineSummery\EEG\EndToEnd\TESST_LOG_PATH',name='E2E_SampleCNN1D_IDSplit')
net=TrainModel(Model,torch.nn.CrossEntropyLoss(),optimizer=torch.optim.Adam(Model.parameters(),lr=0.01))
summary(net,(5,4*2560),device='cpu')
trainer=pl.Trainer(logger=logger)
trainer.fit(net,train_dataloaders=trainDataloader,val_dataloaders=testDataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\loops\utilities.py:72: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name         | Type             | Params
--------------------------------------------------
0 | Model        | simple_cnn1d     | 10.8 K
1 | LossFunction | CrossEntropyLoss | 0     
--------------------------------------------------
10.8 K    Trainable params
0         Non-trainable params
10.8 K    Total params
0.043     Total estimated model params size (MB)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 16, 10240]             576
       BatchNorm1d-2            [-1, 16, 10240]              32
              ReLU-3            [-1, 16, 10240]               0
         MaxPool1d-4             [-1, 16, 5119]               0
            Conv1d-5             [-1, 32, 2560]           1,568
       BatchNorm1d-6             [-1, 32, 2560]              64
              ReLU-7             [-1, 32, 2560]               0
            Conv1d-8             [-1, 64, 1280]           6,208
       BatchNorm1d-9             [-1, 64, 1280]             128
             ReLU-10             [-1, 64, 1280]               0
AdaptiveAvgPool1d-11                [-1, 64, 1]               0
          Flatten-12                   [-1, 64]               0
             ReLU-13                   [-1, 64]               0
           Linear-14                   

Sanity Checking: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\loops\fit_loop.py:281: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=5

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

tensor(1.)
tensor(0.9844)
tensor(0.9141)
tensor(0.9531)
tensor(0.7891)
tensor(0.8906)
tensor(1.)
tensor(0.9688)
tensor(0.8793)


Validation: 0it [00:00, ?it/s]

tensor(0.9922)
tensor(0.9531)
tensor(0.9453)
tensor(0.9531)
tensor(0.8047)
tensor(0.8984)
tensor(0.9766)


g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [90]:
target = torch.tensor([0,0, 1, 2])
preds = torch.tensor([[0.1, 0.9, 0],[0.1, 0.9, 0], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3]])
accuracy(preds, target, task="multiclass", num_classes=3, top_k=2)

target.shape

torch.Size([4])

In [91]:
batch=next(iter(eegDataloader))
x=batch[0].to(torch.float32)
y_hat=net(x)
print(y_hat.shape)
y=batch[1]-1
accuracy(y_hat,y,task='MULTICLASS',num_classes=3)

torch.Size([128, 3])


tensor(0.3594)